## Counterfactual Regret Minimization

### by Lieqiang Guo & Yanting LI

This is the implementation of counterfactual regret minimization on game Kuhn Poker

In [5]:
import random
import numpy as np
from time import time
import matplotlib.pyplot as plt

In [6]:
class Kuhn_Porker:
    PASS = 0
    BET = 1
    NUM_ACTIONS = 2
    node_dict = {}

In [7]:
class Kuhn_Node:
    def __init__(self, n_actions):
        self.infoSet = ''
        self.regretSum = np.zeros(n_actions)
        self.strategy = np.zeros(n_actions)
        self.strategySum = np.zeros(n_actions)
        self.n_actions = n_actions
        
    def getStrategy(self,realizationWeight):
        normalizingSum = 0
        for i in range(self.n_actions):
            if self.regretSum[i] > 0:
                self.strategy[i] = self.regretSum[i]
            else:
                self.strategy[i] = 0
        normalizingSum = np.sum(self.strategy)
        if normalizingSum > 0:
            self.strategy = self.strategy/normalizingSum
        else:
            self.strategy[:] = 1./self.n_actions
        self.strategySum = self.strategySum + realizationWeight*self.strategy
        return self.strategy
    def getAverageStrategy(self):
        avgStrategy = np.zeros(self.n_actions)
        normalizingSum = np.sum(self.strategySum)
        if normalizingSum > 0:
            avgStrategy = self.strategySum/normalizingSum
        else:
            avgStrategy[:] = 1./self.n_actions
        return avgStrategy
    def toString(self):
        avgStrategy = self.getAverageStrategy()
        print("%4s: "%(self.infoSet), "[%.6f, %.6f]" %(avgStrategy[0], avgStrategy[1]))

In [8]:
def train(itr):
    cards = np.array([1,2,3])
    util = 0.
    for i in range(itr):
        np.random.shuffle(cards)
        util = util+cfr(cards[:2], "", 1, 1)
    print("Average game value: ", util/itr)
    nodeKeys = list(Kuhn_Porker.node_dict.keys())
    nodeKeys = sorted(nodeKeys)
    for i in nodeKeys:
        Kuhn_Porker.node_dict[i].toString()
def cfr(cards, history, p0, p1):
    n_actions = 2
    plays = len(history)
    player = plays % 2 # mod()
    opponent = 1-player
    # Return payoff for terminal states
    if plays > 1:
        terminalPass = history[plays-1] == "p"
        double_bet = history[plays-2:plays] =="bb"
        is_player_card_higher = cards[player] > cards[opponent]
        if terminalPass:
            if history == "pp":
                return 1 if is_player_card_higher else -1
            else:
                return 1
        elif(double_bet):
                return 2 if is_player_card_higher else -2
    
    infoSet = str(cards[player]) + history
    
    # Get information set node or create it if nonexistant
    if infoSet not in Kuhn_Porker.node_dict:
        node  = Kuhn_Node(n_actions)
        node.infoSet = infoSet
        Kuhn_Porker.node_dict[infoSet] = node   
    else:
        node = Kuhn_Porker.node_dict[infoSet]
    
    # For each action, recursively call cfr with additional history and probability   
    
    if player == 0:
        realizationWeight = p0
    else:
        realizationWeight = p1
    strategy  = node.getStrategy(realizationWeight)
    util = np.zeros(n_actions)
    nodeUtil = 0
    actions = ['p', 'b']
    for i in range(n_actions):
        nextHistory = history + actions[i]
        if player == 0:
            util[i] = -cfr(cards, nextHistory, p0*strategy[i], p1)
        else:
            util[i] = -cfr(cards, nextHistory, p0, p1*strategy[i])
        nodeUtil = nodeUtil + strategy[i]*util[i]
    
    # For each action, compute and accumulate counterfactual regret
    for i in range(n_actions):
        regret = util[i] - nodeUtil
        if player == 0:
            node.regretSum[i] = node.regretSum[i] + p1*regret
        else:
            node.regretSum[i] = node.regretSum[i] + p0*regret        
    
    return nodeUtil

In [9]:
Kuhn_Porker.node_dict.clear()
train(100000)

Average game value:  -0.0555975465859
   1:  [0.729327, 0.270673]
  1b:  [0.999985, 0.000015]
  1p:  [0.662886, 0.337114]
 1pb:  [0.999990, 0.000010]
   2:  [0.999952, 0.000048]
  2b:  [0.654307, 0.345693]
  2p:  [0.999860, 0.000140]
 2pb:  [0.391671, 0.608329]
   3:  [0.192483, 0.807517]
  3b:  [0.000015, 0.999985]
  3p:  [0.000030, 0.999970]
 3pb:  [0.000039, 0.999961]
